In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, pystan as ps, pickle
from pylab import plot, show, legend
from functions_garch import *
import arviz as az
import random
import os
import pathlib
from time import time
from scipy.stats import wasserstein_distance

In [30]:
T = 2000
p = 5
q = 3
d = 3
X = npr.randn(d,T)
omega = 1.0
b = npr.randn(d)

def generate_Y(T, omega, rescale, b, X):
    beta = np.ones(p) / (rescale * p)
    alpha = np.ones(q) / (rescale * q)
    sigsq, y = simulate_data(T, omega, beta, alpha, b, X)
    return sigsq, y

In [40]:
# step 0: create multiple datasets 
rescales = [2.00, 3.00, 5.00]
datas = {}
n_exp = len(rescales)
for i in range(n_exp):
    rescale = rescales[i]
    datas[rescale] = {}
    sigsq, y = generate_Y(T, omega, rescale, b, X)
    datas[rescale]['alpha'] = np.ones(q) / (rescale * q)
    datas[rescale]['beta'] = np.ones(p) / (rescale * p)
    datas[rescale]['omega'] = omega
    datas[rescale]['b'] = b
    datas[rescale]['sigsq'], datas[rescale]['y'] = sigsq, y
    datas[rescale]['X'] = X

In [47]:
# step 1: make path and save data
pathlib.Path('multiple_sims').mkdir(parents=True, exist_ok=True)
file = open("multiple_sims/datas.pkl", "wb")
pickle.dump(datas, file)
file.close()

In [48]:
# step 2: do multiple simulations
sm = ps.StanModel(file="garch_WB.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f0ef85016fbe1114f4bfb9978c846a08 NOW.
In file included from /var/folders/vm/bnpn0t152gn54b0_nhp78xf40000gp/T/pystan_5wvqf7so/stanfit4anon_model_f0ef85016fbe1114f4bfb9978c846a08_3167031423563654957.cpp:771:
In file included from /opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944:
/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
/var/folders/vm/bnpn0t152gn54b0_nhp78xf40000gp/T/pystan_5wvqf7so/stanfit4anon_model_f0ef85016fbe1114f4bfb9978c846a08_31

In [53]:
# step 2.a: def simulation functions
n_chains = 1
n_iter = 1000

def full_mcmc(rescale, n_iter=500):
    
    # extract data
    print("Doing Full MCMC for rescale="+str(rescale))
    y = datas[rescale]['y']
    X = datas[rescale]['X'] 
    data = dict(T=T, p=p, q=q, r=max(p,q), d=d, y=y, X=X, power=1.)
    
    # MCMC
    start = time()
    fit = sm.sampling(data=data, thin=1, n_jobs=8, chains=n_chains, init="random", iter=n_iter)
    mle = sm.optimizing(data=data)
    print(round((time()-start)/60,2), "minutes to run")
    
    # make path
    rescale_str = str(rescale)
    path = 'multiple_sims/'+ rescale_str
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    
    # save
    print(round((time()-start)/60,2), "minutes to run")
    trace = fit.extract()
    file = open(path+"/full_mcmc.pkl", "wb")
    pickle.dump(trace, file)
    file.close()
    file = open(path+"/full_mle.pkl", "wb")
    pickle.dump(mle, file)
    file.close()


In [61]:
def dc_mcmc(rescale,n_iter=500,m=10):
    m = 10
    rescale_str = str(rescale)
    tstarts = np.arange(m).astype(int)
    tends = 1 + tstarts
    tstarts *= int(T/m)
    tends *= int(T/m)
    print("Doing DC MCMC for alpha="+rescale_str)
    for i in range(m) :
        tstart, tend = tstarts[i], tends[i]
        
        # extract data
        y = datas[rescale]['y']
        X = datas[rescale]['X'] 
        data = dict(T=tend-tstart, p=p, q=q, r=max(p,q), d=d, y=y[tstart:tend], X=X[:,tstart:tend], power=T/(tend-tstart))
        
        # MCMC
        fit = sm.sampling(data=data, thin=1, n_jobs=8, chains=n_chains, init="random", iter=n_iter)
        trace = fit.extract()
        
        # make path
        path = 'multiple_sims/'+rescale_str
        pathlib.Path(path).mkdir(parents=True, exist_ok=True)
        
        # save files
        file = open(path+"/mcmc_wb_chunk"+str(i+1)+".pkl", "wb")
        pickle.dump(trace, file)
        file.close()
        
        mle = sm.optimizing(data=data)
        file = open(path+"/mle_chunk"+str(i+1)+".pkl", "wb")
        pickle.dump(mle, file)
        file.close()

In [63]:
# step 3: run multi sims
n_iter = 1000
for rescale in rescales:
    full_mcmc(rescale, n_iter)
    dc_mcmc(rescale, n_iter, m=10)

Doing Full MCMC for rescale=2.0


Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'garch_WB.stan' at line 39)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'garch_WB.stan' at line 39)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting initial value:
  Log probability evaluates to log(0), i.e. negative infinity.
  Stan can't start sampling from this initial value.
Rejecting ini


Gradient evaluation took 0.000401 seconds
1000 transitions using 10 leapfrog steps per transition would take 4.01 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
Iteration: 300 / 1000 [ 30%]  (Warmup)
Iteration: 400 / 1000 [ 40%]  (Warmup)
Iteration: 500 / 1000 [ 50%]  (Warmup)
Iteration: 501 / 1000 [ 50%]  (Sampling)
Iteration: 600 / 1000 [ 60%]  (Sampling)
Iteration: 700 / 1000 [ 70%]  (Sampling)
Iteration: 800 / 1000 [ 80%]  (Sampling)
Iteration: 900 / 1000 [ 90%]  (Sampling)
Iteration: 1000 / 1000 [100%]  (Sampling)

 Elapsed Time: 13.388 seconds (Warm-up)
               11.9562 seconds (Sampling)
               25.3442 seconds (Total)

Initial log joint probability = -86509
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      19      -5496.42      0.457103       37.2148      0.5496           1       23   
    Iter      



Gradient evaluation took 0.000396 seconds
1000 transitions using 10 leapfrog steps per transition would take 3.96 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
Iteration: 300 / 1000 [ 30%]  (Warmup)
Iteration: 400 / 1000 [ 40%]  (Warmup)
Iteration: 500 / 1000 [ 50%]  (Warmup)
Iteration: 501 / 1000 [ 50%]  (Sampling)
Iteration: 600 / 1000 [ 60%]  (Sampling)
Iteration: 700 / 1000 [ 70%]  (Sampling)
Iteration: 800 / 1000 [ 80%]  (Sampling)
Iteration: 900 / 1000 [ 90%]  (Sampling)
Iteration: 1000 / 1000 [100%]  (Sampling)

 Elapsed Time: 11.6572 seconds (Warm-up)
               10.3188 seconds (Sampling)
               21.976 seconds (Total)

Initial log joint probability = -61730.5
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      19      -5950.39     0.0981292       6.75294           1           1       21   
    Iter   

...

 -3404.24     0.0491725       2.02568      0.6618      0.6618      139   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     139      -3404.06      0.713567       1.45334           1           1      163   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     159      -3404.01       2.56138       1.31679      0.3649           1      186   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     169         -3404       0.24102    0.00678978           1           1      196   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance

Gradient evaluation took 0.000395 seconds
1000 transitions using 10 leapfrog steps per transition would take 3.95 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
It

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'garch_WB.stan' at line 39)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     139      -3145.39     0.0473565       1.96274           1           1      153   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     159      -3145.28       2.61474       3.78025           1           1      174   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     179      -3145.27       3.99316      0.947709           1           1      195   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     187      -3145.27     0.0794546     0.0902342       0.464       0.464      204   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance

Gradient evaluation took 0.00039 seconds
1000 transitions using 10 leapfrog steps per transition would take 3.9 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (War

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'garch_WB.stan' at line 39)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     128      -3319.86     0.0112042     0.0700934      0.6827      0.6827      149   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance

Gradient evaluation took 0.000409 seconds
1000 transitions using 10 leapfrog steps per transition would take 4.09 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
Iteration: 300 / 1000 [ 30%]  (Warmup)
Iteration: 400 / 1000 [ 40%]  (Warmup)
Iteration: 500 / 1000 [ 50%]  (Warmup)
Iteration: 501 / 1000 [ 50%]  (Sampling)
Iteration: 600 / 1000 [ 60%]  (Sampling)
Iteration: 700 / 1000 [ 70%]  (Sampling)
Iteration: 800 / 1000 [ 80%]  (Sampling)
Iteration: 900 / 1000 [ 90%]  (Sampling)
Iteration: 1000 / 1000 [100%]  (Sampling)

 Elapsed Time: 11.3421 seconds (Warm-up)
               6.01469 seconds (Sampling)
    